In [1]:
from pyneurovault import api
import os
from glob import glob
from os.path import join as opj
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import shutil
from random import sample

/Users/egermani/miniforge3/envs/workEnv/lib/python3.9/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


# Fetching NeuroVault 

In [2]:
# Using NeuroVault API, get metadata about all images available in NeuroVault
if not os.path.exists('../data/original/neurovault_api_request_images.csv'):
    df_images = api.get_images(limit=1000) # Search NeuroVault database for all images and 
    # load metadata in a Pandas DataFrame
    df_images.to_csv('../data/original/neurovault_api_request_images.csv')
else:
    print('Dataframe already exists.')
    df_images = pd.read_csv('../data/original/neurovault_api_request_images.csv')

Dataframe already exists.


/var/folders/k4/cck9mrt578q044z6x6zmsnc400m_jt/T/ipykernel_47246/815255208.py:8: DtypeWarning: Columns (7,8,9,10,12,14,18,19,23,24,26,27,28,33,39,40,41,43,47,50,51,52,54,55,56,62,63,64,67,151,152,154,1153,1156,1157,1158,1167,1168,1170,1171,1172,1175,1178,1179,1180,1181,1182,1183) have mixed types. Specify dtype option on import or set low_memory=False.
  df_images = pd.read_csv('../data/original/neurovault_api_request_images.csv')


In [3]:
# Using NeuroVault API, get metadata about all collections available in NeuroVault
if not os.path.exists('../data/original/neurovault_api_request_collections.csv'):
	df_collec = api.get_collections(limit = 10000) # Search NeuroVault database for all collections and 
    #load metadata in a Pandas DataFrame
	df_collec.to_csv('../data/original/neurovault_api_request_collections.csv')
else:
	df_collec = pd.read_csv('../data/original/neurovault_api_request_collections.csv')

/var/folders/k4/cck9mrt578q044z6x6zmsnc400m_jt/T/ipykernel_47246/975620588.py:7: DtypeWarning: Columns (24,26,32,40,51,52,53,54,55,56,57,58,59,60,61,62,63,67,68,69,70,71,72,73,74,76,77,81,89,97,98,99,100,101,102,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  df_collec = pd.read_csv('../data/original/neurovault_api_request_collections.csv')


In [6]:
id_list = []
if os.path.exists(opj('../data/original/NeuroVault_dataset/neurovault_dataset.txt')):
    # If file containing ids of selected maps exists, read it and store the list.
    with open(opj('../data/original/NeuroVault_dataset/neurovault_dataset.txt'), "r") as file:
        lines = file.readlines()
        for line in lines:
            id_list.append(line[:-1])
    file.close()

In [11]:
done = [200008,200109,200210,200311,200513,200614,200917,201111,201414,201515,201717,201818,202113,202719,202820,203418,203923,204016,204218,204319,204420,204521,204622,205119,205220,205725,205826,206222,206323,206525,206727,206828,206929,207123,207426,208024,208125,208226,208327,209127,209228,209329,209531,209834,209935,210011,210112,210415,210617,211114,211215,211316,211417,211619,211720,211821,211922,212015,212116,212217,212318,212419,212823,213017,213421,213522,214019,214221,214423,214524,214726,217126,217429,219231,220721,221218,221319,223929,224022,227432,227533,228434,231928,233326,236130,237334,238033,239136,239944,245333,246133,248238,248339,249947,250427,250932,251833,255639,255740,256540,257542,257845,257946,263436,268749,268850,270332,274542,275645,280739,281135,283543,284646,285345,285446,286347,286650,287248,289555,290136,293748,295146,297655,298051,298455,299154,299760,300618,300719,303119,303624,304020,304727,305830,307127,308129,308331,309636,310621,311320,314225,316633,316835,317332,318637,320826,321323,322224,325129,329440,329844,330324,333330,334635,336841,339847,341834,342129,346137,346945,348545,349244,350330,351938,352132,352738,353740,355239,356948,358144,360030,361234,361941,362034,365343,366042,366446,368753,371843,376247,377451,378756,378857,379657,380036,381038,381543,382242,385046,385450,386250,387959,389357,390645,391748,392447,392750,393247,393550,394956,395251,395756,395958,397154,397760,397861,401422,406432,406836,412528,413934,414229,415837,419239,421226,422632,424939,429040,432332,433839,436239,436845,441939,445543,448347,449753,453441,453542,454140,456346,459453,461743,462139,463040,465852,467351,468050,469961,473952,475855,479762,480141,481042,481951,485757,486759,492754,495255,497865,499566,500222,506234,510225,510326,512835,513130,513736,516742,517239,518746,519647,519950,520228,521331,522434,523032,524135,525541,529549,529953,530635,531536,536647,540436,541640,541943,545345,547046,548250,549757,552241,552544,553344,555348,555651,555954,557857,558657,558960,559053,559457,561242,561444,561949,562345,562446,565452,566454,567052,567759,567961,568963,569965,570243,571144,572045,573249,573451,576255,578057,578158,579665,579867,580044,580650,580751,581349,581450,583858,585256,585862,586460,587664,588565,589567,590047,592455,594156,597869,598568,599065,599469,599671,601127,604537,609143,611938,613235,613538,614439,615744,616645,617748,618952,623844,626648,627549,633847,634748,635245,638049,644044,644246,645450,654552,654754,656253,656657,657659,660951,663755,665254,667056,671855,672756,673455,675661,677766,677968,698168,700634,701535,713239,729254,744553,753150,757764,759869,760551,761957,763557,765056,765864,766563,767464,769064,770352,771354,773257,774663,779370,782561,783462,784565,786569,788674,788876,789373,792564,792766,814548,814649,849264,852455,856463,857263,859671,861456,865363,867468,869472,870861,871762,871964,872158,872562,872764,873968,877168,877269,878776,878877,880157,882161,884064,885975,886674,887373,888678,889579,891667,894067,894673,894774,896778,896879,898176,899885,901038]

disp = [200008,200109,200210,200311,200513,200614,200917,201111,201414,201515,201717,201818,202113,202719,202820,203418,203923,204016,204218,204319,204420,204521,204622,205119,205220,205725,205826,206222,206323,206525,206727,206828,206929,207123,207426,208024,208125,208226,208327,209127,209228,209329,209531,209834,209935,210011,210112,210415,210617,211114,211215,211316,211417,211619,211720,211821,211922,212015,212116,212217,212318,212419,212823,213017,213421,213522,214019,214221,214423,214524,214726,217126,217429,219231,220721,221218,221319,223929,224022,227432,227533,228434,231928,233326,236130,237334,238033,239136,239944,245333,246133,248238,248339,249947,250427,250932,251833,255639,255740,256540,257542,257845,257946,263436,268749,268850,270332,274542,275645,280739,280941,281135,283543,284646,285345,285446,286347,286650,287248,289555,290136,293748,295146,297655,298051,298455,299154,299760,300618,300719,303119,303624,304020,304727,305830,307127,308129,308331,309636,310621,311320,314225,316633,316835,317332,318637,320826,321323,322224,325129,329440,329844,330324,333330,334635,336841,339847,341834,342129,346137,346945,348545,349244,350330,351938,352132,352738,353740,355239,356948,358144,360030,361234,361941,362034,365343,366042,366446,368551,368753,371843,376247,377451,378756,378857,379657,380036,381038,381543,382242,385046,385450,386250,387959,389357,390645,391748,392447,392750,393247,393550,394956,395251,395756,395958,397154,397760,397861,401422,406432,406836,412528,413934,414229,415837,419239,421226,422632,424939,429040,432332,433839,436239,436845,441939,445543,448347,449753,453441,453542,454140,456346,459453,461743,462139,463040,465852,467351,468050,469961,473952,475855,479762,480141,481042,481951,485757,486759,492754,495255,497865,499566,500222,506234,510225,510326,512835,513130,513736,516742,517239,518746,519647,519950,520228,521331,522434,523032,524135,525541,529549,529953,530635,531536,536647,540436,541640,541943,545345,547046,548250,549757,552241,552544,553344,555348,555651,555954,557857,558657,558960,559053,559457,561242,561444,561949,562345,562446,565452,566454,567052,567759,567961,568963,569965,570243,571144,572045,573249,573451,576255,578057,578158,579665,579867,580044,580347,580650,580751,581349,581450,583858,585256,585862,586460,587664,588565,589567,590047,592455,594156,597869,598568,599065,599469,599671,601127,604537,609143,611938,613235,613538,614439,615441,615744,616645,617748,618952,620434,622236,623137,623844,626648,627549,627852,628248,633847,634748,635245,638049,644044,644246,645450,645551,647858,654350,654552,654754,656253,656657,657659,660951,662551,663755,664757,665254,667056,668361,671855,672756,673455,675661,677766,677968,679568,679770,680250,680452,680957,683256,685058,686969,687163,688569,689470,690152,692964,693461,693764,694362,695768,698168,700634,701535,702133,704238,705341,706040,707749,709551,713239,715041,715647,715950,720337,723141,724446,725751,727553,727654,728454,729254,729557,731140,732243,734045,734247,735148,737960,742549,744553,748258,748662,749058,749361,751348,751550,753150,753251,756055,757764,759869,760551,761957,763557,765056,765864,766563,767464,769064,770352,771354,773257,774663,779370,782561,783462,784565,786569,788674,788876,789373,792564,792766,792867,793465,800941,802844,803240,804646,809252,810439,810843,812746,814548,814649,815247,816653,818455,818859,820745,822244,825048,825553,825654,826353,826454,827052,828862,832651,833148,833249,835657,837560,837964,841349,843151,844961,845458,849264,849971,852455,856463,856766,856968,857263,859671,861456,865363,867468,869472,870861,871762,871964,872158,872562,872764,873968,877168,877269,878776,878877,880157,882161,884064,885975,886674,887373,888678,889579,891667,894067,894673,894774,896778,896879,898176,899885,901038,901139,901442,902242,904044,905147,907656,908860,910241,910443,911849,912447,917255,917558,919966,922854,923755,926862,927359,929464,930449,932554,933253,937160,942658,943862,947668,951457,952863,953764,955465,957974,958976,959574,962058,965367,965771,966975,969476,970764,971160,972566,973770,978578,979984,983773,984472,987074,987983,989987,990366,991267,992673,992774,993675,994273,995174,996782]

In [14]:
len(done)

439

In [16]:
len(disp) + 484

1080

In [13]:
len([i for i in disp if i not in done])

157

In [10]:
subject_level_df=df_images[df_images['image_id'].isin([int(i) for i in id_list])]
subject_level_df.groupby(['analysis_level']).nunique()

,Unnamed: 0,url,image_id,file,collection,collection_id,file_size,cognitive_paradigm_cogatlas,cognitive_paradigm_cogatlas_id,cognitive_contrast_cogatlas,...,ICV,SDS-BMI,Site,Actual paradigm label,subject_id,Stimuli,group 1,group 2,group 1 group size,group 2 group size
analysis_level,,,,,,,,,,,,,,,,,,,,,
group,14740,14740,14740,14740,4291,4291,12089,217,217,200,...,0,0,0,0,0,3,2,1,8,3
single-subject,13792,13792,13792,13792,231,231,10165,66,66,11,...,0,45,3,0,0,0,0,0,0,0


# Selection of NeuroVault dataset

In [4]:
error_images = [51429, 51430, 67793, 110481, 110482, 110483, 110484, 110487, 110488, 110489, 110490, 113776, 115008, 
133840, 133841, 133842, 133843, 133844, 133845, 133846, 133847, 133848, 133849, 133850, 306407, 306408, 306409,
306410, 306411, 306412, 306413, 360472, 394776, 394777, 394778, 394779, 350627, 1076,
13165, 14181,14182,14183,14184,14185,14186,14187,14188,14189,14190,14191,14192,14193,14194,14195,14196,14197,14198,
14199,14200,14272,15995,15996,15999,16000,18511,18512,18513,18514,18515,18516,18517,18518,18519,18520,18521,18522,
18523,18524,18525,18643,18644,18669,18670,20696,23569,23687,24091,28871,28872,28886,28937,28938,28957,28958,39055,
39879, 43806, 44483, 44485, 50315, 51432, 51433,52580,52620,52927,53140,53142,53144,53146,53148,53150,53163,53229,
53493,53495,53497,56157,58233,58234,58235,58236,58237,58344,58348,59158,59159,60072,60330,61079,61482,61483,61484,
61485,61486,61487,61488,61489,62502,62704,62705,63130,64373,64374,64376,64391,64766,65080,65085,65140,65215,65221,
65227,65417,65418,65426,67916,67917,68058,68087,68088,68089,68090,68091,100342,124714,124718,124719,124720,124721,
124722,124723,124724,124725,124726,124736,124755,124756,124757,124758,124759,124760,124761,124911,124912,128534,
128535,128536,128537,128538,128539,128540,128541,128542,128543,128544,128545,128546,128547,128548,128549,128550,
128551,128552,128553,135787,135788,135789,304534,304536,381325,381326,381327,381328,381329,381330,381331,381332,
381333,381334,381335,381336,381337,381338,381339,381340,381341,381342,381343,381344,406412,406417,513197,513207] 
# Error when trying to download these images --> Corrupted ?


# Retrieval of selected images
id_list = []
if os.path.exists(opj('../data/original/NeuroVault_dataset/neurovault_dataset.txt')):
    # If file containing ids of selected maps exists, read it and store the list.
    with open(opj('../data/original/NeuroVault_dataset/neurovault_dataset.txt'), "r") as file:
        lines = file.readlines()
        for line in lines:
            id_list.append(line[:-1])
    file.close()

else:
    # If file doesn't exist, make selection based on metadata using dataframe.
    for i in df_images.index:
        if (df_images.loc[i]['analysis_level'] == 'group' or df_images.loc[i]['analysis_level'] == 'single-subject')\
        and df_images.loc[i]['is_valid'] and not df_images.loc[i]['not_mni'] \
        and (df_images.loc[i]['map_type'] == 'T map' or df_images.loc[i]['map_type'] == 'Z map') \
        and not df_images.loc[i]['is_thresholded'] \
        and df_images.loc[i]['modality'] == 'fMRI-BOLD' and df_images.loc[i]['image_type'] == 'statistic_map' \
        and not ("Set" in df_images.loc[i]['file'] and "mean" in df_images.loc[i]['file']) \
        and df_images.loc[i]['image_id'] not in error_images:
            id_list.append(str(df_images.loc[i]['image_id']))

    subject_level_df = df_images[df_images['image_id'].isin([int(i) for i in id_list])]
    subject_level_df = subject_level_df[subject_level_df['collection_id'].isin(df_collec['collection_id']\
                                        [~df_collec['name'].str.contains('temporary')].tolist())]

    id_list = [str(i) for i in subject_level_df['image_id'].tolist()]
    
    # Store selected ids in txt file to avoid the time consuming task of selection. 
    with open(opj('../data/original/NeuroVault_dataset/neurovault_dataset.txt'), "w") as file:
        for ids in id_list:
            file.write(str(ids))
            file.write('\n')
    file.close()

print('NeuroVault dataset:', len(id_list), 'images')

NeuroVault dataset: 28532 images


In [5]:
# Splitting the dataset into TRAIN and TEST 
# If file available, read the IDs 
if os.path.exists(opj('../data/original/NeuroVault_dataset', 'train_neurovault_dataset.txt')) and \
    os.path.exists(opj('../data/original/NeuroVault_dataset', 'test_neurovault_dataset.txt')):
    
    global_train_df = []
    with open(opj('../data/original/NeuroVault_dataset', 'train_neurovault_dataset.txt'), "r") as file:
        lines = file.readlines()
        for line in lines:
            global_train_df.append(line[:-1])
    file.close() 
    
    global_test_df = []
    with open(opj('../data/original/NeuroVault_dataset', 'test_neurovault_dataset.txt'), "r") as file:
        lines = file.readlines()
        for line in lines:
            global_test_df.append(line[:-1])
    file.close() 

# If files not available, split the data and store the ids in files
else: 
    
    global_df = []
    with open(opj('../data/original/NeuroVault_dataset', 'neurovault_dataset.txt'), "r") as file:
        lines = file.readlines()
        for line in lines:
            global_df.append(line[:-1])
    file.close() 
    
    global_train_df, global_test_df = train_test_split(global_df, 
                                                       test_size = 0.2, train_size = 0.8)
    
    
    with open(opj('../data/original/NeuroVault_dataset/train_neurovault_dataset.txt'), "w") as file:
        for ids in global_train_df:
            file.write(str(ids))
            file.write('\n')
    file.close()
    
    with open(opj('../data/original/NeuroVault_dataset/test_neurovault_dataset.txt'), "w") as file:
        for ids in global_test_df:
            file.write(str(ids))
            file.write('\n')
    file.close()

In [6]:
print('Training set:', len(global_train_df))
print('Test set:', len(global_test_df))

Training set: 22772
Test set: 5760


# Selection of HCP datasets

In [7]:
# Finding IDs of HCP images stored in NeuroVault
if not os.path.exists('../data/original/HCP_dataset/hcp_dataset.txt'): 
    df_hcp = df_images[df_images['collection_id']==4337]
    
    id_list = df_hcp['image_id'].tolist()
    with open(opj('../data/original/HCP_dataset/hcp_dataset.txt'), "w") as file:
        for ids in id_list:
            file.write(str(ids))
            file.write('\n')
    file.close()
    
else: 
    df_hcp = df_images[df_images['collection_id']==4337]
    
    id_list = []
    with open(opj('../data/original/HCP_dataset/hcp_dataset.txt'), "r") as file:
        lines = file.readlines()
        for line in lines:
            id_list.append(line[:-1])
    file.close()
    
    
print('HCP dataset:', len(id_list), 'images')

HCP dataset: 18070 images


In [8]:
# Finding useful metadata and store them in the label file
if not os.path.exists('../data/original/HCP_dataset/hcp_dataset_labels.csv'):
    
    df_hcp['subject'] = [0 for i in range(18070)]

    for i in df_hcp['file'].to_list():
        df_hcp['subject'][df_hcp['file']==i] = df_hcp['name'][df_hcp['file']==i].tolist()[0][0:6]

    df_hcp = df_hcp[['image_id', 'subject', 'cognitive_paradigm_cogatlas', 'task', 'contrast_definition']]
    df_hcp.rename(columns = {'contrast_definition':'contrast'}, inplace = True)

    df_hcp.to_csv('../data/original/HCP_dataset/hcp_dataset_labels.csv')
    
else:
    df_hcp = pd.read_csv('../data/original/HCP_dataset/hcp_dataset_labels.csv')

In [9]:
# TRAIN / VALID SPLIT 
# If file already exists, read ids
subs = 'hcp_dataset'
data_dir = '../data/original/HCP_dataset'
if os.path.exists(opj(data_dir, 'train_' + str(subs) + '.txt')):
    train_df = []
    # If file containing ids of selected maps exists, read it and store the list.
    with open(opj(data_dir, 'train_' + str(subs) + '.txt'), "r") as file:
        lines = file.readlines()
        for line in lines:
            train_df.append(line[:-1])
    file.close()
    
    valid_df = []
    # If file containing ids of selected maps exists, read it and store the list.
    with open(opj(data_dir, 'valid_' + str(subs) + '.txt'), "r") as file:
        lines = file.readlines()
        for line in lines:
            valid_df.append(line[:-1])
    file.close()

# If file doesn't exist, perform split
else:
    global_df = []
    with open(opj(data_dir, f'{subs}.txt'), "r") as file:
        lines = file.readlines()
        for line in lines:
            global_df.append(line[:-1])
    file.close()
    
    global_subs = np.unique(df_hcp['subject'].tolist())
    print(len(global_subs))
    
    train_subs, valid_subs = train_test_split(global_subs, 
                                            test_size = 0.5, train_size = 0.5)
    
    print(len(train_subs))
    print(len(valid_subs))
    
    train_df = df_hcp['image_id'][df_hcp['subject'].isin(train_subs)]
    valid_df = df_hcp['image_id'][df_hcp['subject'].isin(valid_subs)]
        
    with open(opj(data_dir, 'train_' + str(subs) + '.txt'), "w") as file:
        for ids in train_df:
            file.write(str(ids))
            file.write('\n')
    file.close()
    
    with open(opj(data_dir, 'valid_' + str(subs) + '.txt'), "w") as file:
        for ids in valid_df:
            file.write(str(ids))
            file.write('\n')
    file.close()

print('Training set:', len(train_df))
print('Valid set:', len(valid_df))

Training set: 9017
Valid set: 9053


## HCP global subsets of different sample sizes

In [10]:
train_subs = np.unique(df_hcp['subject'][df_hcp['image_id'].isin([int(i) for i in train_df])].tolist())
valid_subs = np.unique(df_hcp['subject'][df_hcp['image_id'].isin([int(i) for i in valid_df])].tolist())
print(len(train_subs))
for n in [200, 100, 50]:
    print('Sample', n)
    if not os.path.exists(opj(data_dir, 'train_hcp_dataset_'+ str(n) + '.txt')):
        train_subs = np.random.choice(train_subs, n, replace=False)
        print(len(train_subs))
        train_df = df_hcp['image_id'][df_hcp['subject'].isin(train_subs)]
        print('Train:', len(train_df))
        
        valid_subs = np.random.choice(valid_subs, n, replace=False)
        print(len(valid_subs))
        valid_df = df_hcp['image_id'][df_hcp['subject'].isin(valid_subs)]
        print('Valid:', len(valid_df))
        
        with open(opj(data_dir, f'train_hcp_dataset_{n}.txt'), "w") as file:
            for ids in train_df:
                file.write(str(ids))
                file.write('\n')
        file.close()

        with open(opj(data_dir, f'valid_hcp_dataset_{n}.txt'), "w") as file:
            for ids in valid_df:
                file.write(str(ids))
                file.write('\n')
        file.close()
        
    else:
        train_df = []
        with open(opj(data_dir, f'train_hcp_dataset_{n}.txt'), "r") as file:
            lines = file.readlines()
            for line in lines:
                train_df.append(line[:-1])
        file.close()
        print('Train:', len(train_df))
        
        valid_df = []
        with open(opj(data_dir, f'valid_hcp_dataset_{n}.txt'), "r") as file:
            lines = file.readlines()
            for line in lines:
                valid_df.append(line[:-1])
        file.close()
        print('Valid:', len(valid_df))
        

    df_hcp.to_csv(opj(data_dir, f'hcp_dataset_{n}_labels.csv'))

393
Sample 200
Train: 4590
Valid: 4591
Sample 100
Train: 2300
Valid: 2300
Sample 50
Train: 1150
Valid: 1150


## Small HCP dataset for One contrast task classification 

In [11]:
# TRAIN / VALID SPLIT 
# If file already exists, read ids
subs = 'small_hcp_dataset'
data_dir = '../data/original/HCP_dataset'
df_hcp = pd.read_csv('../data/original/HCP_dataset/hcp_dataset_labels.csv')

if not os.path.exists(opj(data_dir, f'{subs}_labels.csv')):
    small_hcp_df = df_hcp[df_hcp['contrast'].isin(['2BKPLACE', 'FACES', 'PUNISH', 'REL' ,'RH', 'STORY' ,'TOM'])]
    small_hcp_df.to_csv(opj(data_dir, f'{subs}_labels.csv'))

else:
    small_hcp_df = pd.read_csv(opj(data_dir, f'{subs}_labels.csv'))

if os.path.exists(opj(data_dir, 'train_' + str(subs) + '.txt')):
    train_df = []
    # If file containing ids of selected maps exists, read it and store the list.
    with open(opj(data_dir, 'train_' + str(subs) + '.txt'), "r") as file:
        lines = file.readlines()
        for line in lines:
            train_df.append(line[:-1])
    file.close()
    
    valid_df = []
    # If file containing ids of selected maps exists, read it and store the list.
    with open(opj(data_dir, 'valid_' + str(subs) + '.txt'), "r") as file:
        lines = file.readlines()
        for line in lines:
            valid_df.append(line[:-1])
    file.close()

# If file doesn't exist, find images
else:
    train_id = []
    # If file containing ids of selected maps exists, read it and store the list.
    with open(opj(data_dir, 'train_hcp_dataset.txt'), "r") as file:
        lines = file.readlines()
        for line in lines:
            train_id.append(int(line[:-1]))
    file.close()
    
    valid_id = []
    # If file containing ids of selected maps exists, read it and store the list.
    with open(opj(data_dir, 'valid_hcp_dataset.txt'), "r") as file:
        lines = file.readlines()
        for line in lines:
            valid_id.append(int(line[:-1]))
    file.close()
    
    train_df = small_hcp_df['image_id'][small_hcp_df['image_id'].isin(train_id)]
    
    valid_df = small_hcp_df['image_id'][small_hcp_df['image_id'].isin(valid_id)]
    
    with open(opj(data_dir, 'train_' + str(subs) + '.txt'), "w") as file:
        for ids in train_df:
            file.write(str(ids))
            file.write('\n')
    file.close()
    
    with open(opj(data_dir, 'valid_' + str(subs) + '.txt'), "w") as file:
        for ids in valid_df:
            file.write(str(ids))
            file.write('\n')
    file.close()
    
print('Training set:', len(train_df))
print('Valid set:', len(valid_df))

Training set: 2355
Valid set: 2361


In [12]:
train_subs = np.unique(small_hcp_df['subject'][small_hcp_df['image_id'].isin([int(i) for i in train_df])].tolist())
valid_subs = np.unique(small_hcp_df['subject'][small_hcp_df['image_id'].isin([int(i) for i in valid_df])].tolist())
print(len(train_subs))
for n in [200, 100, 50]:
    print('Sample', n)
    if not os.path.exists(opj(data_dir, 'train_small_hcp_dataset_'+ str(n) + '.txt')):
        train_subs = np.random.choice(train_subs, n, replace=False)
        train_df = small_hcp_df['image_id'][small_hcp_df['subject'].isin(train_subs)]
        print('Train:', len(train_df))
        
        valid_subs = np.random.choice(valid_subs, n, replace=False)
        valid_df = small_hcp_df['image_id'][small_hcp_df['subject'].isin(valid_subs)]
        print('Valid:', len(valid_df))
        
        with open(opj(data_dir, f'train_small_hcp_dataset_{n}.txt'), "w") as file:
            for ids in train_df:
                file.write(str(ids))
                file.write('\n')
        file.close()

        with open(opj(data_dir, f'valid_small_hcp_dataset_{n}.txt'), "w") as file:
            for ids in valid_df:
                file.write(str(ids))
                file.write('\n')
        file.close()
        
    else:
        train_df = []
        with open(opj(data_dir, f'train_small_hcp_dataset_{n}.txt'), "r") as file:
            lines = file.readlines()
            for line in lines:
                train_df.append(line[:-1])
        file.close()
        print('Train:', len(train_df))
        
        valid_df = []
        with open(opj(data_dir, f'valid_small_hcp_dataset_{n}.txt'), "r") as file:
            lines = file.readlines()
            for line in lines:
                valid_df.append(line[:-1])
        file.close()
        print('Valid:', len(valid_df))
        

    small_hcp_df.to_csv(opj(data_dir, f'small_hcp_dataset_{n}_labels.csv'))

393
Sample 200
Train: 1198
Valid: 1200
Sample 100
Train: 598
Valid: 600
Sample 50
Train: 300
Valid: 300


In [13]:
# KFOLD CV 
import random
import math
data_dir = '../data/original/HCP_dataset'
df_hcp = pd.read_csv('../data/original/HCP_dataset/hcp_dataset_labels.csv')
df_small_hcp = pd.read_csv('../data/original/HCP_dataset/small_hcp_dataset_labels.csv')

subs = 'hcp_dataset_50'
train_df = []
# If file containing ids of selected maps exists, read it and store the list.
with open(opj(data_dir, 'train_' + str(subs) + '.txt'), "r") as file:
    lines = file.readlines()
    for line in lines:
        train_df.append(int(line[:-1]))
file.close()
train_subs = np.unique(df_hcp['subject'][df_hcp['image_id'].isin(train_df)].tolist())

random.shuffle(train_subs)
n_val = math.floor(len(train_subs)/5)

test_folds_50 = [train_subs[i:i + n_val].tolist() for i in range(0, len(train_subs), n_val)]
train_folds_50 = [[subs for subs in train_subs if subs not in test_folds_50[i]] for i in range(5)]

train_folds_50_id = [df_hcp['image_id'][df_hcp['subject'].isin(train_folds_50[i])].tolist() for i in range(5)]
test_folds_50_id = [df_hcp['image_id'][df_hcp['subject'].isin(test_folds_50[i])].tolist() for i in range(5)]

for i in range(5):
    with open(opj(data_dir, f'train_perf_hcp_dataset_50_fold_{i}.txt'), "w") as file:
        print(len(train_folds_50_id[i]))
        for ids in train_folds_50_id[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()
        
    with open(opj(data_dir, f'test_perf_hcp_dataset_50_fold_{i}.txt'), "w") as file:
        print(len(test_folds_50_id[i]))
        for ids in test_folds_50_id[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()

train_folds_50_id_small = [df_small_hcp['image_id'][df_small_hcp['subject'].isin(train_folds_50[i])].tolist() for i in range(5)]
test_folds_50_id_small = [df_small_hcp['image_id'][df_small_hcp['subject'].isin(test_folds_50[i])].tolist() for i in range(5)]


for i in range(5):
    with open(opj(data_dir, f'train_perf_small_hcp_dataset_50_fold_{i}.txt'), "w") as file:
        print(len(train_folds_50_id_small[i]))
        for ids in train_folds_50_id_small[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()
        
    with open(opj(data_dir, f'test_perf_small_hcp_dataset_50_fold_{i}.txt'), "w") as file:
        print(len(test_folds_50_id_small[i]))
        for ids in test_folds_50_id_small[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()

subs = 'hcp_dataset_100'
train_df = []
# If file containing ids of selected maps exists, read it and store the list.
with open(opj(data_dir, 'train_' + str(subs) + '.txt'), "r") as file:
    lines = file.readlines()
    for line in lines:
        train_df.append(int(line[:-1]))
file.close()
train_subs = np.unique(df_hcp['subject'][df_hcp['image_id'].isin(train_df)][~df_hcp['subject'].isin([item for sublist in test_folds_50 for item in sublist])].tolist())
random.shuffle(train_subs)
n_val = math.floor(len(train_subs)/5)

test_folds = [train_subs[i:i + n_val].tolist() for i in range(0, len(train_subs), n_val)]
test_folds_100 = [test_folds_50[i] + test_folds[i] for i in range(5)]
train_subs = np.unique(df_hcp['subject'][df_hcp['image_id'].isin(train_df)].tolist())
train_folds_100 = [[subs for subs in train_subs if subs not in test_folds_100[i]] for i in range(5)]

train_folds_100_id = [df_hcp['image_id'][df_hcp['subject'].isin(train_folds_100[i])].tolist() for i in range(5)]
test_folds_100_id = [df_hcp['image_id'][df_hcp['subject'].isin(test_folds_100[i])].tolist() for i in range(5)]

for i in range(5):
    with open(opj(data_dir, f'train_perf_hcp_dataset_100_fold_{i}.txt'), "w") as file:
        for ids in train_folds_100_id[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()
        
    with open(opj(data_dir, f'test_perf_hcp_dataset_100_fold_{i}.txt'), "w") as file:
        for ids in test_folds_100_id[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()

train_folds_100_id_small = [df_small_hcp['image_id'][df_small_hcp['subject'].isin(train_folds_100[i])].tolist() for i in range(5)]
test_folds_100_id_small = [df_small_hcp['image_id'][df_small_hcp['subject'].isin(test_folds_100[i])].tolist() for i in range(5)]


for i in range(5):
    with open(opj(data_dir, f'train_perf_small_hcp_dataset_100_fold_{i}.txt'), "w") as file:
        for ids in train_folds_100_id_small[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()
        
    with open(opj(data_dir, f'test_perf_small_hcp_dataset_100_fold_{i}.txt'), "w") as file:
        for ids in test_folds_100_id_small[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()

subs = 'hcp_dataset_200'
train_df = []
# If file containing ids of selected maps exists, read it and store the list.
with open(opj(data_dir, 'train_' + str(subs) + '.txt'), "r") as file:
    lines = file.readlines()
    for line in lines:
        train_df.append(int(line[:-1]))
file.close()
train_subs = np.unique(df_hcp['subject'][df_hcp['image_id'].isin(train_df)][~df_hcp['subject'].isin([item for sublist in test_folds_100 for item in sublist])].tolist())
random.shuffle(train_subs)
n_val = math.floor(len(train_subs)/5)

test_folds = [train_subs[i:i + n_val].tolist() for i in range(0, len(train_subs), n_val)]
test_folds_200 = [test_folds_100[i] + test_folds[i] for i in range(5)]
train_subs = np.unique(df_hcp['subject'][df_hcp['image_id'].isin(train_df)].tolist())
train_folds_200 = [[subs for subs in train_subs if subs not in test_folds_200[i]] for i in range(5)]

train_folds_200_id = [df_hcp['image_id'][df_hcp['subject'].isin(train_folds_200[i])].tolist() for i in range(5)]
test_folds_200_id = [df_hcp['image_id'][df_hcp['subject'].isin(test_folds_200[i])].tolist() for i in range(5)]

for i in range(5):
    with open(opj(data_dir, f'train_perf_hcp_dataset_200_fold_{i}.txt'), "w") as file:
        for ids in train_folds_200_id[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()
        
    with open(opj(data_dir, f'test_perf_hcp_dataset_200_fold_{i}.txt'), "w") as file:
        for ids in test_folds_200_id[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()

train_folds_200_id_small = [df_small_hcp['image_id'][df_small_hcp['subject'].isin(train_folds_200[i])].tolist() for i in range(5)]
test_folds_200_id_small = [df_small_hcp['image_id'][df_small_hcp['subject'].isin(test_folds_200[i])].tolist() for i in range(5)]


for i in range(5):
    with open(opj(data_dir, f'train_perf_small_hcp_dataset_200_fold_{i}.txt'), "w") as file:
        for ids in train_folds_200_id_small[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()
        
    with open(opj(data_dir, f'test_perf_small_hcp_dataset_200_fold_{i}.txt'), "w") as file:
        for ids in test_folds_200_id_small[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()

subs = 'hcp_dataset'
train_df = []
# If file containing ids of selected maps exists, read it and store the list.
with open(opj(data_dir, 'train_' + str(subs) + '.txt'), "r") as file:
    lines = file.readlines()
    for line in lines:
        train_df.append(int(line[:-1]))
file.close()
train_subs = np.unique(df_hcp['subject'][df_hcp['image_id'].isin(train_df)][~df_hcp['subject'].isin([item for sublist in test_folds_200 for item in sublist])].tolist())
random.shuffle(train_subs)
n_val = math.floor(len(train_subs)/5)

test_folds = [train_subs[i:i + n_val].tolist() for i in range(0, len(train_subs), n_val)]
test_folds_all = [test_folds_200[i] + test_folds[i] for i in range(5)]
train_subs = np.unique(df_hcp['subject'][df_hcp['image_id'].isin(train_df)].tolist())
train_folds_all = [[subs for subs in train_subs if subs not in test_folds_all[i]] for i in range(5)]

train_folds_all_id = [df_hcp['image_id'][df_hcp['subject'].isin(train_folds_all[i])].tolist() for i in range(5)]
test_folds_all_id = [df_hcp['image_id'][df_hcp['subject'].isin(test_folds_all[i])].tolist() for i in range(5)]

for i in range(5):
    with open(opj(data_dir, f'train_perf_hcp_dataset_fold_{i}.txt'), "w") as file:
        for ids in train_folds_all_id[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()
        
    with open(opj(data_dir, f'test_perf_hcp_dataset_fold_{i}.txt'), "w") as file:
        for ids in test_folds_all_id[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()

train_folds_all_id_small = [df_small_hcp['image_id'][df_small_hcp['subject'].isin(train_folds_all[i])].tolist() for i in range(5)]
test_folds_all_id_small = [df_small_hcp['image_id'][df_small_hcp['subject'].isin(test_folds_all[i])].tolist() for i in range(5)]


for i in range(5):
    with open(opj(data_dir, f'train_perf_small_hcp_dataset_fold_{i}.txt'), "w") as file:
        for ids in train_folds_all_id_small[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()
        
    with open(opj(data_dir, f'test_perf_small_hcp_dataset_fold_{i}.txt'), "w") as file:
        for ids in test_folds_all_id_small[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()


920
230
920
230
920
230
920
230
920
230
240
60
240
60
240
60
240
60
240
60


In [14]:
# KFOLD CV 
import random
import math
data_dir = '../data/original/HCP_dataset'
df_hcp = pd.read_csv('../data/original/HCP_dataset/hcp_dataset_labels.csv')
df_small_hcp = pd.read_csv('../data/original/HCP_dataset/small_hcp_dataset_labels.csv')

subs = 'hcp_dataset_50'
train_df = []
# If file containing ids of selected maps exists, read it and store the list.
with open(opj(data_dir, 'valid_' + str(subs) + '.txt'), "r") as file:
    lines = file.readlines()
    for line in lines:
        train_df.append(int(line[:-1]))
file.close()
train_subs = np.unique(df_hcp['subject'][df_hcp['image_id'].isin(train_df)].tolist())

random.shuffle(train_subs)
n_val = math.floor(len(train_subs)/5)

test_folds_50 = [train_subs[i:i + n_val].tolist() for i in range(0, len(train_subs), n_val)]
train_folds_50 = [[subs for subs in train_subs if subs not in test_folds_50[i]] for i in range(5)]

train_folds_50_id = [df_hcp['image_id'][df_hcp['subject'].isin(train_folds_50[i])].tolist() for i in range(5)]
test_folds_50_id = [df_hcp['image_id'][df_hcp['subject'].isin(test_folds_50[i])].tolist() for i in range(5)]

for i in range(5):
    with open(opj(data_dir, f'train_hp_hcp_dataset_50_fold_{i}.txt'), "w") as file:
        print(len(train_folds_50_id[i]))
        for ids in train_folds_50_id[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()
        
    with open(opj(data_dir, f'test_hp_hcp_dataset_50_fold_{i}.txt'), "w") as file:
        print(len(test_folds_50_id[i]))
        for ids in test_folds_50_id[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()

train_folds_50_id_small = [df_small_hcp['image_id'][df_small_hcp['subject'].isin(train_folds_50[i])].tolist() for i in range(5)]
test_folds_50_id_small = [df_small_hcp['image_id'][df_small_hcp['subject'].isin(test_folds_50[i])].tolist() for i in range(5)]


for i in range(5):
    with open(opj(data_dir, f'train_hp_small_hcp_dataset_50_fold_{i}.txt'), "w") as file:
        print(len(train_folds_50_id_small[i]))
        for ids in train_folds_50_id_small[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()
        
    with open(opj(data_dir, f'test_hp_small_hcp_dataset_50_fold_{i}.txt'), "w") as file:
        print(len(test_folds_50_id_small[i]))
        for ids in test_folds_50_id_small[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()

subs = 'hcp_dataset_100'
train_df = []
# If file containing ids of selected maps exists, read it and store the list.
with open(opj(data_dir, 'valid_' + str(subs) + '.txt'), "r") as file:
    lines = file.readlines()
    for line in lines:
        train_df.append(int(line[:-1]))
file.close()
train_subs = np.unique(df_hcp['subject'][df_hcp['image_id'].isin(train_df)][~df_hcp['subject'].isin([item for sublist in test_folds_50 for item in sublist])].tolist())
random.shuffle(train_subs)
n_val = math.floor(len(train_subs)/5)

test_folds = [train_subs[i:i + n_val].tolist() for i in range(0, len(train_subs), n_val)]
test_folds_100 = [test_folds_50[i] + test_folds[i] for i in range(5)]
train_subs = np.unique(df_hcp['subject'][df_hcp['image_id'].isin(train_df)].tolist())
train_folds_100 = [[subs for subs in train_subs if subs not in test_folds_100[i]] for i in range(5)]

train_folds_100_id = [df_hcp['image_id'][df_hcp['subject'].isin(train_folds_100[i])].tolist() for i in range(5)]
test_folds_100_id = [df_hcp['image_id'][df_hcp['subject'].isin(test_folds_100[i])].tolist() for i in range(5)]

for i in range(5):
    with open(opj(data_dir, f'train_hp_hcp_dataset_100_fold_{i}.txt'), "w") as file:
        for ids in train_folds_100_id[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()
        
    with open(opj(data_dir, f'test_hp_hcp_dataset_100_fold_{i}.txt'), "w") as file:
        for ids in test_folds_100_id[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()

train_folds_100_id_small = [df_small_hcp['image_id'][df_small_hcp['subject'].isin(train_folds_100[i])].tolist() for i in range(5)]
test_folds_100_id_small = [df_small_hcp['image_id'][df_small_hcp['subject'].isin(test_folds_100[i])].tolist() for i in range(5)]


for i in range(5):
    with open(opj(data_dir, f'train_hp_small_hcp_dataset_100_fold_{i}.txt'), "w") as file:
        for ids in train_folds_100_id_small[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()
        
    with open(opj(data_dir, f'test_hp_small_hcp_dataset_100_fold_{i}.txt'), "w") as file:
        for ids in test_folds_100_id_small[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()

subs = 'hcp_dataset_200'
train_df = []
# If file containing ids of selected maps exists, read it and store the list.
with open(opj(data_dir, 'valid_' + str(subs) + '.txt'), "r") as file:
    lines = file.readlines()
    for line in lines:
        train_df.append(int(line[:-1]))
file.close()
train_subs = np.unique(df_hcp['subject'][df_hcp['image_id'].isin(train_df)][~df_hcp['subject'].isin([item for sublist in test_folds_100 for item in sublist])].tolist())
random.shuffle(train_subs)
n_val = math.floor(len(train_subs)/5)

test_folds = [train_subs[i:i + n_val].tolist() for i in range(0, len(train_subs), n_val)]
test_folds_200 = [test_folds_100[i] + test_folds[i] for i in range(5)]
train_subs = np.unique(df_hcp['subject'][df_hcp['image_id'].isin(train_df)].tolist())
train_folds_200 = [[subs for subs in train_subs if subs not in test_folds_200[i]] for i in range(5)]

train_folds_200_id = [df_hcp['image_id'][df_hcp['subject'].isin(train_folds_200[i])].tolist() for i in range(5)]
test_folds_200_id = [df_hcp['image_id'][df_hcp['subject'].isin(test_folds_200[i])].tolist() for i in range(5)]

for i in range(5):
    with open(opj(data_dir, f'train_hp_hcp_dataset_200_fold_{i}.txt'), "w") as file:
        for ids in train_folds_200_id[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()
        
    with open(opj(data_dir, f'test_hp_hcp_dataset_200_fold_{i}.txt'), "w") as file:
        for ids in test_folds_200_id[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()

train_folds_200_id_small = [df_small_hcp['image_id'][df_small_hcp['subject'].isin(train_folds_200[i])].tolist() for i in range(5)]
test_folds_200_id_small = [df_small_hcp['image_id'][df_small_hcp['subject'].isin(test_folds_200[i])].tolist() for i in range(5)]


for i in range(5):
    with open(opj(data_dir, f'train_hp_small_hcp_dataset_200_fold_{i}.txt'), "w") as file:
        for ids in train_folds_200_id_small[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()
        
    with open(opj(data_dir, f'test_hp_small_hcp_dataset_200_fold_{i}.txt'), "w") as file:
        for ids in test_folds_200_id_small[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()

subs = 'hcp_dataset'
train_df = []
# If file containing ids of selected maps exists, read it and store the list.
with open(opj(data_dir, 'valid_' + str(subs) + '.txt'), "r") as file:
    lines = file.readlines()
    for line in lines:
        train_df.append(int(line[:-1]))
file.close()
train_subs = np.unique(df_hcp['subject'][df_hcp['image_id'].isin(train_df)][~df_hcp['subject'].isin([item for sublist in test_folds_200 for item in sublist])].tolist())
random.shuffle(train_subs)
n_val = math.floor(len(train_subs)/5)

test_folds = [train_subs[i:i + n_val].tolist() for i in range(0, len(train_subs), n_val)]
test_folds_all = [test_folds_200[i] + test_folds[i] for i in range(5)]
train_subs = np.unique(df_hcp['subject'][df_hcp['image_id'].isin(train_df)].tolist())
train_folds_all = [[subs for subs in train_subs if subs not in test_folds_all[i]] for i in range(5)]

train_folds_all_id = [df_hcp['image_id'][df_hcp['subject'].isin(train_folds_all[i])].tolist() for i in range(5)]
test_folds_all_id = [df_hcp['image_id'][df_hcp['subject'].isin(test_folds_all[i])].tolist() for i in range(5)]

for i in range(5):
    with open(opj(data_dir, f'train_hp_hcp_dataset_fold_{i}.txt'), "w") as file:
        for ids in train_folds_all_id[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()
        
    with open(opj(data_dir, f'test_hp_hcp_dataset_fold_{i}.txt'), "w") as file:
        for ids in test_folds_all_id[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()

train_folds_all_id_small = [df_small_hcp['image_id'][df_small_hcp['subject'].isin(train_folds_all[i])].tolist() for i in range(5)]
test_folds_all_id_small = [df_small_hcp['image_id'][df_small_hcp['subject'].isin(test_folds_all[i])].tolist() for i in range(5)]


for i in range(5):
    with open(opj(data_dir, f'train_hp_small_hcp_dataset_fold_{i}.txt'), "w") as file:
        for ids in train_folds_all_id_small[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()
        
    with open(opj(data_dir, f'test_hp_small_hcp_dataset_fold_{i}.txt'), "w") as file:
        for ids in test_folds_all_id_small[i]:
            file.write(str(ids))
            file.write('\n')
    file.close()


920
230
920
230
920
230
920
230
920
230
240
60
240
60
240
60
240
60
240
60


# Selection of BrainPedia datasets

## Large BrainPedia

In [15]:
df_bp = df_images[df_images['collection_id']==1952]

tags_to_rm = []
for t in np.unique(df_bp['tags']).tolist():
    if len(df_bp[df_bp['tags']==t]) <= 30:
        tags_to_rm.append(t)

df_bp = df_bp[~df_bp['tags'].isin(tags_to_rm)]

In [16]:
if not os.path.exists('../data/original/BrainPedia_dataset/bp_dataset.txt'):
    id_list = df_bp['image_id'].tolist()
    with open(opj('../data/original/BrainPedia_dataset/bp_dataset.txt'), "w") as file:
        for ids in id_list:
            file.write(str(ids))
            file.write('\n')
    file.close()
    
else: 
    id_list = []
    with open(opj('../data/original/BrainPedia_dataset/bp_dataset.txt'), "r") as file:
        lines = file.readlines()
        for line in lines:
            id_list.append(line[:-1])
    file.close()
    
print('BrainPedia dataset:', len(id_list), 'images')

BrainPedia dataset: 6448 images


In [17]:
if not os.path.exists('../data/original/BrainPedia_dataset/bp_dataset_labels.csv'):    
    subjects = [name.split('_')[-1] for name in df_bp['name'].tolist()]
    studies = [name.split('_')[0] for name in df_bp['name'].tolist()]
    tasks = [name.split('_')[1] for name in df_bp['name'].tolist()]

    df_bp = df_bp[['image_id', 'tags', 'name']]

    df_bp['subject'] = subjects
    df_bp['study'] = studies
    df_bp['task'] = tasks
    
    df_bp.to_csv('../data/original/BrainPedia_dataset/bp_dataset_labels.csv')
    
else:
    df_bp = pd.read_csv('../data/original/BrainPedia_dataset/bp_dataset_labels.csv')

In [18]:
# TRAIN / VALID SPLIT 
# If file already exists, read ids
subs = 'bp_dataset'
data_dir = '../data/original/BrainPedia_dataset'
if os.path.exists(opj(data_dir, 'train_' + str(subs) + '.txt')):
    train_df = []
    # If file containing ids of selected maps exists, read it and store the list.
    with open(opj(data_dir, 'train_' + str(subs) + '.txt'), "r") as file:
        lines = file.readlines()
        for line in lines:
            train_df.append(line[:-1])
    file.close()
    
    valid_df = []
    # If file containing ids of selected maps exists, read it and store the list.
    with open(opj(data_dir, 'valid_' + str(subs) + '.txt'), "r") as file:
        lines = file.readlines()
        for line in lines:
            valid_df.append(line[:-1])
    file.close()

# If file doesn't exist, perform split
else:
    global_df = []
    with open(opj(data_dir, f'{subs}.txt'), "r") as file:
        lines = file.readlines()
        for line in lines:
            global_df.append(line[:-1])
    file.close()
    
    train_df = pd.DataFrame()
    valid_df = pd.DataFrame()
    
    for study in np.unique(df_bp['study'].tolist()):
        subjects = np.unique(df_bp['subject'][df_bp['study']==study])
        if len(subjects) > 1:
            train_subs, valid_subs = train_test_split(subjects, 
                                                test_size = 0.5, train_size = 0.5)
            train_df = train_df.append(df_bp[df_bp['study']==study]\
                                                 [df_bp['subject'].isin(train_subs)])

            valid_df = valid_df.append(df_bp[df_bp['study']==study]\
                                                 [df_bp['subject'].isin(valid_subs)])
            
        else: 
            train_df = train_df.append(df_bp[df_bp['study']==study]\
                                                 [df_bp['subject'].isin(subjects)])

    with open(opj(data_dir, 'train_' + str(subs) + '.txt'), "w") as file:
        for ids in train_df['image_id']:
            file.write(str(ids))
            file.write('\n')
    file.close()
    
    with open(opj(data_dir, 'valid_' + str(subs) + '.txt'), "w") as file:
        for ids in valid_df['image_id']:
            file.write(str(ids))
            file.write('\n')
    file.close()
    
print('Training set:', len(train_df))
print('Valid set:', len(valid_df))

Training set: 3179
Valid set: 3269


## Small BrainPedia

In [19]:
subs = 'small_bp_dataset'
data_dir = '../data/original/BrainPedia_dataset'

train_df = []
# If file containing ids of selected maps exists, read it and store the list.
with open(opj(data_dir, 'train_bp_dataset.txt'), "r") as file:
    lines = file.readlines()
    for line in lines:
        train_df.append(line[:-1])
file.close()

valid_df = []
# If file containing ids of selected maps exists, read it and store the list.
with open(opj(data_dir, 'valid_bp_dataset.txt'), "r") as file:
    lines = file.readlines()
    for line in lines:
        valid_df.append(line[:-1])
file.close()
if not os.path.exists(opj(data_dir, f'{subs}_labels.csv')):
    studies = df_bp.groupby(['study']).nunique()['subject'] > 20

    small_bp_studies = [studies.index.tolist()[i] for i in range(len(studies)) if studies[i]]

    small_bp_df = df_bp[df_bp['study'].isin(small_bp_studies)]
    
    train_small_bp = small_bp_df[small_bp_df['image_id'].isin([int(i) for i in train_df])]
    valid_small_bp = small_bp_df[small_bp_df['image_id'].isin([int(i) for i in valid_df])]
    
    train_small_bp_2 = pd.DataFrame()
    valid_small_bp_2 = pd.DataFrame()
    
    for study in np.unique(small_bp_df['study'].tolist()):
        print('Study:', study)
        subjects = np.unique(train_small_bp['subject'][train_small_bp['study']==study])
        subjects_to_keep = np.random.choice(subjects, 10, replace=False)
        train_small_bp_2 = train_small_bp_2.append(small_bp_df[small_bp_df['study']==study]\
                                             [small_bp_df['subject'].isin(subjects_to_keep)])
        
        subjects = np.unique(valid_small_bp['subject'][valid_small_bp['study']==study])
        subjects_to_keep = np.random.choice(subjects, 10, replace=False)
        valid_small_bp_2 = valid_small_bp_2.append(small_bp_df[small_bp_df['study']==study]\
                                             [small_bp_df['subject'].isin(subjects_to_keep)])
        
    
    train_df = train_small_bp_2['image_id']
    valid_df = valid_small_bp_2['image_id']
    
    small_bp_df_2 = train_small_bp_2.append(valid_small_bp_2)
    
    with open(opj(data_dir, 'train_' + str(subs) + '.txt'), "w") as file:
        for ids in train_df:
            file.write(str(ids))
            file.write('\n')
    file.close()
    
    with open(opj(data_dir, 'valid_' + str(subs) + '.txt'), "w") as file:
        for ids in valid_df:
            file.write(str(ids))
            file.write('\n')
    file.close()
        
    small_bp_df_2.to_csv(opj(data_dir, f'{subs}_labels.csv'))

else:
    small_bp_df_2 = pd.read_csv(opj(data_dir, f'{subs}_labels.csv'))
    
    train_df = []
    # If file containing ids of selected maps exists, read it and store the list.
    with open(opj(data_dir, 'train_' + str(subs) + '.txt'), "r") as file:
        lines = file.readlines()
        for line in lines:
            train_df.append(line[:-1])
    file.close()
    
    valid_df = []
    # If file containing ids of selected maps exists, read it and store the list.
    with open(opj(data_dir, 'valid_' + str(subs) + '.txt'), "r") as file:
        lines = file.readlines()
        for line in lines:
            valid_df.append(line[:-1])
    file.close()
    

print('Training set:', len(train_df))
print('Valid set:', len(valid_df))

Training set: 917
Valid set: 927


In [20]:
data_dir = '../data/original/BrainPedia_dataset'
df_bp = pd.read_csv('../data/original/BrainPedia_dataset/bp_dataset_labels.csv')
df_small_bp = pd.read_csv('../data/original/BrainPedia_dataset/small_bp_dataset_labels.csv')

In [21]:
import random
import math
data_dir = '../data/original/BrainPedia_dataset'
df_bp = pd.read_csv('../data/original/BrainPedia_dataset/bp_dataset_labels.csv')
df_small_bp = pd.read_csv('../data/original/BrainPedia_dataset/small_bp_dataset_labels.csv')

subs = 'small_bp_dataset'
train_df = []
# If file containing ids of selected maps exists, read it and store the list.
with open(opj(data_dir, 'train_' + str(subs) + '.txt'), "r") as file:
    lines = file.readlines()
    for line in lines:
        train_df.append(int(line[:-1]))
file.close()

train_folds_sbp_id = []
test_folds_sbp_id = []
test_folds_sbp = []
train_folds_sbp = []

for s, studies in enumerate(np.unique(df_small_bp['study'].tolist())):
    train_subs = np.unique(df_small_bp['subject'][df_small_bp['study']==studies][df_small_bp['image_id'].isin(train_df)].tolist())

    random.shuffle(train_subs)
    n_val = math.floor(len(train_subs)/5)

    test_folds_sbp.append([train_subs[i:i + n_val].tolist() for i in range(0, len(train_subs), n_val)])
    train_folds_sbp.append([[subs for subs in train_subs if subs not in test_folds_sbp[s][i]] for i in range(5)])

    train_folds_sbp_id.append([df_small_bp['image_id'][df_small_bp['study']==studies][df_small_bp['subject'].isin(train_folds_sbp[s][i])].tolist() for i in range(5)])
    test_folds_sbp_id.append([df_small_bp['image_id'][df_small_bp['study']==studies][df_small_bp['subject'].isin(test_folds_sbp[s][i])].tolist() for i in range(5)])


for i in range(5):
    fold_length_tr = 0
    fold_length_te = 0
    with open(opj(data_dir, f'train_perf_small_bp_dataset_fold_{i}.txt'), "w") as file:
        for s in range(len(np.unique(df_small_bp['study'].tolist()))):
            fold_length_tr += len(train_folds_sbp_id[s][i])
            for ids in train_folds_sbp_id[s][i]:
                file.write(str(ids))
                file.write('\n')
    file.close()

    with open(opj(data_dir, f'test_perf_small_bp_fold_{i}.txt'), "w") as file:
        for s in range(len(np.unique(df_small_bp['study'].tolist()))):
            fold_length_te += len(test_folds_sbp_id[s][i])
            for ids in test_folds_sbp_id[s][i]:
                file.write(str(ids))
                file.write('\n')
    file.close()
    
    print(f'Small BP fold {i}: {fold_length_tr} train, {fold_length_te} test')


subs = 'bp_dataset'
train_df = []
# If file containing ids of selected maps exists, read it and store the list.
with open(opj(data_dir, 'train_' + str(subs) + '.txt'), "r") as file:
    lines = file.readlines()
    for line in lines:
        train_df.append(int(line[:-1]))
file.close()

train_folds_bp_id = []
test_folds_bp_id = []
train_folds_bp = []
test_folds_bp = []

# For each study of BrainPedia
for s, studies in enumerate(np.unique(df_bp['study'].tolist())):
    # Check if this study is also part of small Brainpedia 
    # If so, find the list of subjects that are already included in the folds of small BrainPedia 
    # These will be the base of the folds of BrainPedia for this study
    if studies in np.unique(df_small_bp['study'].tolist()):
        s_bis = np.unique(df_small_bp['study'].tolist()).tolist().index(studies)
        # Train subs corresponds to the subjects that are part of the study but not in small brainpedia 
        train_subs = np.unique(df_bp['subject'][df_bp['study']==studies][df_bp['image_id'].isin(train_df)][~df_bp['subject'].isin([item for sublist in test_folds_sbp[s_bis] for item in sublist])].tolist())
        # Shuffle these subjects 
        random.shuffle(train_subs)
        n_val = math.floor(len(train_subs)/5)
        # If there are more than 5 subjects --> do a classical CV
        if len(train_subs) >= 5:
            test_folds = [train_subs[i:i + n_val].tolist() for i in range(0, len(train_subs), n_val)]
            test_folds_bp.append([test_folds_sbp[s_bis][i] + test_folds[i] for i in range(5)])
            train_folds_bp.append([train_folds_sbp[s_bis][i] + [subs for subs in train_subs if subs not in test_folds_bp[s][i]] for i in range(5)])
            train_folds_bp_id.append([df_bp['image_id'][df_bp['study']==studies][df_bp['subject'].isin(train_folds_bp[s][i])].tolist() for i in range(5)])
            test_folds_bp_id.append([df_bp['image_id'][df_bp['study']==studies][df_bp['subject'].isin(test_folds_bp[s][i])].tolist() for i in range(5)])
        else:
            test_folds = [[train_subs[i]] for i in range(len(train_subs))] + [[] for i in range(5-len(train_subs))]
            test_folds_bp.append([test_folds_sbp[s_bis][i] + test_folds[i] for i in range(5)])
            train_folds_bp.append([train_folds_sbp[s_bis][i] + [subs for subs in train_subs if subs not in test_folds_bp[s][i]] for i in range(5)])
            train_folds_bp_id.append([df_bp['image_id'][df_bp['study']==studies][df_bp['subject'].isin(train_folds_bp[s][i])].tolist() for i in range(5)])
            test_folds_bp_id.append([df_bp['image_id'][df_bp['study']==studies][df_bp['subject'].isin(test_folds_bp[s][i])].tolist() for i in range(5)])

    else:
        train_subs = np.unique(df_bp['subject'][df_bp['study']==studies][df_bp['image_id'].isin(train_df)].tolist())
        random.shuffle(train_subs)
        n_val = math.floor(len(train_subs)/5)

        if len(train_subs) >= 5:
            test_folds = [train_subs[i:i + n_val].tolist() for i in range(0, len(train_subs), n_val)]
            test_folds_bp.append([test_folds[i] for i in range(5)])
            train_folds_bp.append([[subs for subs in train_subs if subs not in test_folds_bp[s][i]] for i in range(5)])
            train_folds_bp_id.append([df_bp['image_id'][df_bp['study']==studies][df_bp['subject'].isin(train_folds_bp[s][i])].tolist() for i in range(5)])
            test_folds_bp_id.append([df_bp['image_id'][df_bp['study']==studies][df_bp['subject'].isin(test_folds_bp[s][i])].tolist() for i in range(5)])
        else:
            test_folds = [[train_subs[i]] for i in range(len(train_subs))] + [[] for i in range(5-len(train_subs))]
            test_folds_bp.append([test_folds[i] for i in range(5)])
            train_folds_bp.append([[subs for subs in train_subs if subs not in test_folds_bp[s][i]] for i in range(5)])
            train_folds_bp_id.append([df_bp['image_id'][df_bp['study']==studies][df_bp['subject'].isin(train_folds_bp[s][i])].tolist() for i in range(5)])
            test_folds_bp_id.append([df_bp['image_id'][df_bp['study']==studies][df_bp['subject'].isin(test_folds_bp[s][i])].tolist() for i in range(5)])

for i in range(5):
    fold_length_tr = 0
    fold_length_te = 0
    with open(opj(data_dir, f'train_perf_bp_dataset_fold_{i}.txt'), "w") as file:
        for s in range(len(np.unique(df_bp['study'].tolist()))):
            fold_length_tr += len(train_folds_bp_id[s][i])
            for ids in train_folds_bp_id[s][i]:
                file.write(str(ids))
                file.write('\n')
    file.close()

    with open(opj(data_dir, f'test_perf_bp_fold_{i}.txt'), "w") as file:
        for s in range(len(np.unique(df_bp['study'].tolist()))):
            fold_length_te += len(test_folds_bp_id[s][i])
            for ids in test_folds_bp_id[s][i]:
                file.write(str(ids))
                file.write('\n')
    file.close()
    
    print(f'BP fold {i}: {fold_length_tr} train, {fold_length_te} test')

Small BP fold 0: 734 train, 183 test
Small BP fold 1: 734 train, 183 test
Small BP fold 2: 729 train, 188 test
Small BP fold 3: 736 train, 181 test
Small BP fold 4: 735 train, 182 test
BP fold 0: 2597 train, 582 test
BP fold 1: 2600 train, 579 test
BP fold 2: 2593 train, 586 test
BP fold 3: 2617 train, 562 test
BP fold 4: 2614 train, 565 test


In [22]:
import random
import math
data_dir = '../data/original/BrainPedia_dataset'
df_bp = pd.read_csv('../data/original/BrainPedia_dataset/bp_dataset_labels.csv')
df_small_bp = pd.read_csv('../data/original/BrainPedia_dataset/small_bp_dataset_labels.csv')

subs = 'small_bp_dataset'
train_df = []
# If file containing ids of selected maps exists, read it and store the list.
with open(opj(data_dir, 'valid_' + str(subs) + '.txt'), "r") as file:
    lines = file.readlines()
    for line in lines:
        train_df.append(int(line[:-1]))
file.close()

train_folds_sbp_id = []
test_folds_sbp_id = []
test_folds_sbp = []
train_folds_sbp = []

for s, studies in enumerate(np.unique(df_small_bp['study'].tolist())):
    train_subs = np.unique(df_small_bp['subject'][df_small_bp['study']==studies][df_small_bp['image_id'].isin(train_df)].tolist())

    random.shuffle(train_subs)
    n_val = math.floor(len(train_subs)/5)

    test_folds_sbp.append([train_subs[i:i + n_val].tolist() for i in range(0, len(train_subs), n_val)])
    train_folds_sbp.append([[subs for subs in train_subs if subs not in test_folds_sbp[s][i]] for i in range(5)])

    train_folds_sbp_id.append([df_small_bp['image_id'][df_small_bp['study']==studies][df_small_bp['subject'].isin(train_folds_sbp[s][i])].tolist() for i in range(5)])
    test_folds_sbp_id.append([df_small_bp['image_id'][df_small_bp['study']==studies][df_small_bp['subject'].isin(test_folds_sbp[s][i])].tolist() for i in range(5)])

for i in range(5):
    fold_length_tr = 0
    fold_length_te = 0
    with open(opj(data_dir, f'train_hp_small_bp_dataset_fold_{i}.txt'), "w") as file:
        for s in range(len(np.unique(df_small_bp['study'].tolist()))):
            fold_length_tr += len(train_folds_sbp_id[s][i])
            for ids in train_folds_sbp_id[s][i]:
                file.write(str(ids))
                file.write('\n')
    file.close()

    with open(opj(data_dir, f'test_hp_small_bp_fold_{i}.txt'), "w") as file:
        for s in range(len(np.unique(df_small_bp['study'].tolist()))):
            fold_length_te += len(test_folds_sbp_id[s][i])
            for ids in test_folds_sbp_id[s][i]:
                file.write(str(ids))
                file.write('\n')
    file.close()
    
    print(f'Small BP fold {i}: {fold_length_tr} train, {fold_length_te} test')


subs = 'bp_dataset'
train_df = []
# If file containing ids of selected maps exists, read it and store the list.
with open(opj(data_dir, 'valid_' + str(subs) + '.txt'), "r") as file:
    lines = file.readlines()
    for line in lines:
        train_df.append(int(line[:-1]))
file.close()

train_folds_bp_id = []
test_folds_bp_id = []
train_folds_bp = []
test_folds_bp = []

# For each study of BrainPedia
for s, studies in enumerate(np.unique(df_bp['study'].tolist())):
    # Check if this study is also part of small Brainpedia 
    # If so, find the list of subjects that are already included in the folds of small BrainPedia 
    # These will be the base of the folds of BrainPedia for this study
    if studies in np.unique(df_small_bp['study'].tolist()):
        s_bis = np.unique(df_small_bp['study'].tolist()).tolist().index(studies)
        # Train subs corresponds to the subjects that are part of the study but not in small brainpedia 
        train_subs = np.unique(df_bp['subject'][df_bp['study']==studies][df_bp['image_id'].isin(train_df)][~df_bp['subject'].isin([item for sublist in test_folds_sbp[s_bis] for item in sublist])].tolist())
        # Shuffle these subjects 
        random.shuffle(train_subs)
        n_val = math.floor(len(train_subs)/5)
        # If there are more than 5 subjects --> do a classical CV
        if len(train_subs) >= 5:
            test_folds = [train_subs[i:i + n_val].tolist() for i in range(0, len(train_subs), n_val)]
            test_folds_bp.append([test_folds_sbp[s_bis][i] + test_folds[i] for i in range(5)])
            train_folds_bp.append([train_folds_sbp[s_bis][i] + [subs for subs in train_subs if subs not in test_folds_bp[s][i]] for i in range(5)])
            train_folds_bp_id.append([df_bp['image_id'][df_bp['study']==studies][df_bp['subject'].isin(train_folds_bp[s][i])].tolist() for i in range(5)])
            test_folds_bp_id.append([df_bp['image_id'][df_bp['study']==studies][df_bp['subject'].isin(test_folds_bp[s][i])].tolist() for i in range(5)])
        else:
            test_folds = [[train_subs[i]] for i in range(len(train_subs))] + [[] for i in range(5-len(train_subs))]
            test_folds_bp.append([test_folds_sbp[s_bis][i] + test_folds[i] for i in range(5)])
            train_folds_bp.append([train_folds_sbp[s_bis][i] + [subs for subs in train_subs if subs not in test_folds_bp[s][i]] for i in range(5)])
            train_folds_bp_id.append([df_bp['image_id'][df_bp['study']==studies][df_bp['subject'].isin(train_folds_bp[s][i])].tolist() for i in range(5)])
            test_folds_bp_id.append([df_bp['image_id'][df_bp['study']==studies][df_bp['subject'].isin(test_folds_bp[s][i])].tolist() for i in range(5)])

    else:
        train_subs = np.unique(df_bp['subject'][df_bp['study']==studies][df_bp['image_id'].isin(train_df)].tolist())
        random.shuffle(train_subs)
        n_val = math.floor(len(train_subs)/5)

        if len(train_subs) >= 5:
            test_folds = [train_subs[i:i + n_val].tolist() for i in range(0, len(train_subs), n_val)]
            test_folds_bp.append([test_folds[i] for i in range(5)])
            train_folds_bp.append([[subs for subs in train_subs if subs not in test_folds_bp[s][i]] for i in range(5)])
            train_folds_bp_id.append([df_bp['image_id'][df_bp['study']==studies][df_bp['subject'].isin(train_folds_bp[s][i])].tolist() for i in range(5)])
            test_folds_bp_id.append([df_bp['image_id'][df_bp['study']==studies][df_bp['subject'].isin(test_folds_bp[s][i])].tolist() for i in range(5)])
        else:
            test_folds = [[train_subs[i]] for i in range(len(train_subs))] + [[] for i in range(5-len(train_subs))]
            test_folds_bp.append([test_folds[i] for i in range(5)])
            train_folds_bp.append([[subs for subs in train_subs if subs not in test_folds_bp[s][i]] for i in range(5)])
            train_folds_bp_id.append([df_bp['image_id'][df_bp['study']==studies][df_bp['subject'].isin(train_folds_bp[s][i])].tolist() for i in range(5)])
            test_folds_bp_id.append([df_bp['image_id'][df_bp['study']==studies][df_bp['subject'].isin(test_folds_bp[s][i])].tolist() for i in range(5)])

for i in range(5):
    fold_length_tr = 0
    fold_length_te = 0
    with open(opj(data_dir, f'train_hp_bp_dataset_fold_{i}.txt'), "w") as file:
        for s in range(len(np.unique(df_bp['study'].tolist()))):
            fold_length_tr += len(train_folds_bp_id[s][i])
            for ids in train_folds_bp_id[s][i]:
                file.write(str(ids))
                file.write('\n')
    file.close()

    with open(opj(data_dir, f'test_hp_bp_fold_{i}.txt'), "w") as file:
        for s in range(len(np.unique(df_bp['study'].tolist()))):
            fold_length_te += len(test_folds_bp_id[s][i])
            for ids in test_folds_bp_id[s][i]:
                file.write(str(ids))
                file.write('\n')
    file.close()
    
    print(f'BP fold {i}: {fold_length_tr} train, {fold_length_te} test')

Small BP fold 0: 745 train, 182 test
Small BP fold 1: 743 train, 184 test
Small BP fold 2: 736 train, 191 test
Small BP fold 3: 735 train, 192 test
Small BP fold 4: 749 train, 178 test
BP fold 0: 2673 train, 596 test
BP fold 1: 2668 train, 601 test
BP fold 2: 2663 train, 606 test
BP fold 3: 2675 train, 594 test
BP fold 4: 2684 train, 585 test
